# Homework 1

#### Nawat Ngerncham

In [1]:
import numpy as np
import numpy.random as rd
import pandas as pd

from numpy.random import default_rng
from dataclasses import dataclass
from typing import List, Callable, Tuple, TypeVar


@dataclass
class TSPResult:
    final_route: List[int]
    final_distance: float
    progress: pd.DataFrame


DEFAULT_N_ITERATIONS = 100_000

In [2]:
T = TypeVar("T")


def shuffle(lst: List[T]) -> List[T]:
    rng = default_rng()
    lst_cp = lst.copy()
    rng.shuffle(lst_cp)
    return lst_cp

## 2-OPT

In [3]:
def random_adj_idx(l: int) -> Tuple[int, int]:
    assert l >= 4
    pair1 = rd.randint(l)
    pair2 = rd.randint(l)

    while pair1 - 1 < pair2 < pair1 + 1:
        pair2 = rd.randint(l)

    return pair1, pair2


def two_opt_tsp(cities: List[int],
                path_distance: Callable[[List[int]], float],
                n_iters: int = DEFAULT_N_ITERATIONS) -> TSPResult:
    current_path = cities.copy()
    current_dist = path_distance(current_path)
    progress = [current_dist]

    for _ in range(n_iters):
        new_path = current_path.copy()

        pi1, pi2 = random_adj_idx(len(cities))
        new_path[pi1], new_path[pi2] = current_path[pi2], current_path[pi1]
        new_distance = path_distance(new_path)

        if new_distance < current_dist:
            current_path = new_path
            current_dist = new_distance

        progress.append(current_dist)

    return TSPResult(
        final_route=current_path,
        final_distance=current_dist,
        progress=pd.DataFrame({"distance": progress})
    )

## Random Sampling

In [4]:
def random_sampling_tsp(cities: List[int],
                        path_distance: Callable[[List[int]], float],
                        n_iters: int = DEFAULT_N_ITERATIONS) -> TSPResult:
    current_path = cities.copy()
    current_distance = path_distance(current_path)
    dist_progress = [current_distance]

    for _ in range(n_iters):
        new_path = shuffle(cities.copy())
        new_distance = path_distance(new_path)

        if new_distance < current_distance:
            current_distance = new_distance
            current_path = new_path

        dist_progress.append(current_distance)

    return TSPResult(
        final_route=current_path,
        final_distance=current_distance,
        progress=pd.DataFrame({"distance": dist_progress})
    )

## In-class 2-OPT Example

In [5]:
def in_class_2opt_distance(x: int, y: int):
    dist_matrix = [
        [0, 7, 7, 3, 8],
        [7, 0, 7, 5, 3],
        [7, 7, 0, 8, 5],
        [3, 5, 8, 0, 5],
        [8, 3, 5, 5, 0]
    ]

    return dist_matrix[x][y]


def in_class_path_distance(path: List[int]):
    path_to_use = path + [path[0]]
    return np.sum([in_class_2opt_distance(x, y) for x, y in zip(path_to_use[:-1], path_to_use[1:])])

In [6]:
random_sampling_tsp([i for i in range(5)], in_class_path_distance, n_iters=250)

TSPResult(final_route=[1, 4, 2, 0, 3], final_distance=23, progress=     distance
0          35
1          28
2          28
3          28
4          28
..        ...
246        23
247        23
248        23
249        23
250        23

[251 rows x 1 columns])

In [7]:
two_opt_tsp([i for i in range(5)], in_class_path_distance, n_iters=250)

TSPResult(final_route=[0, 2, 4, 1, 3], final_distance=23, progress=     distance
0          35
1          29
2          28
3          23
4          23
..        ...
246        23
247        23
248        23
249        23
250        23

[251 rows x 1 columns])